In [1]:
# import packages
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings("ignore")

**Find the original data here:**\
https://opendata-downloads.s3.amazonaws.com/opa_properties_public.csv \
**Find the data codebook here**:\
https://metadata.phila.gov/#home/datasetdetails/5543865f20583086178c4ee5/representationdetails/55d624fdad35c7e854cb21a4/?view_287_page=1

In [2]:
# read data
data = pd.read_csv('opa_properties_public.csv', header=0)
print(data.shape)
print(data.columns)
data.head()

(581191, 78)
Index(['objectid', 'assessment_date', 'basements', 'beginning_point',
       'book_and_page', 'building_code', 'building_code_description',
       'category_code', 'category_code_description', 'census_tract',
       'central_air', 'cross_reference', 'date_exterior_condition', 'depth',
       'exempt_building', 'exempt_land', 'exterior_condition', 'fireplaces',
       'frontage', 'fuel', 'garage_spaces', 'garage_type',
       'general_construction', 'geographic_ward', 'homestead_exemption',
       'house_extension', 'house_number', 'interior_condition', 'location',
       'mailing_address_1', 'mailing_address_2', 'mailing_care_of',
       'mailing_city_state', 'mailing_street', 'mailing_zip', 'market_value',
       'market_value_date', 'number_of_bathrooms', 'number_of_bedrooms',
       'number_of_rooms', 'number_stories', 'off_street_open',
       'other_building', 'owner_1', 'owner_2', 'parcel_number', 'parcel_shape',
       'quality_grade', 'recording_date', 'registry_nu

,objectid,assessment_date,basements,beginning_point,book_and_page,building_code,building_code_description,category_code,category_code_description,census_tract,...,unit,utility,view_type,year_built,year_built_estimate,zip_code,zoning,pin,lat,lng
0,96022750,2022-03-08 14:22:35,NaN,SWC REED ST,53764990.0,SR,VACANT LAND RESIDE < ACRE,14,NaN,32.0,...,NaN,NaN,I,NaN,NaN,19146.0,RSA5,1001681390,-75.185501,39.934553
1,96022751,2022-03-08 14:16:10,NaN,SWC REED ST,53764990.0,SR,VACANT LAND RESIDE < ACRE,2,Multi Family,32.0,...,NaN,NaN,I,NaN,NaN,19146.0,RSA5,1001681388,-75.185177,39.934722
2,96022752,2022-03-08 14:14:15,NaN,SWC REED ST,53764990.0,SR,VACANT LAND RESIDE < ACRE,14,NaN,32.0,...,NaN,NaN,I,NaN,NaN,19146.0,RSA5,1001681384,-75.185423,39.934910
3,96022753,2022-03-08 14:08:24,NaN,SWC REED ST,53764990.0,SR,VACANT LAND RESIDE < ACRE,6,Vacant Land,32.0,...,NaN,NaN,I,NaN,NaN,19146.0,RSA5,1001681383,-75.185423,39.934910
4,96022754,2022-03-08 14:14:36,NaN,SWC REED ST,53764990.0,SR,VACANT LAND RESIDE < ACRE,6,Vacant Land,32.0,...,NaN,NaN,I,NaN,NaN,19146.0,RSA5,1001681385,-75.185151,39.934839


### Step 1: select related columns 

In [3]:
# place columns of interest into a list
cols_interest = ['assessment_date', 'basements', 'building_code_description', 'category_code_description','census_tract',
                 'central_air', 'depth', 'exempt_building', 'exempt_land', 'exterior_condition', 
                 'fireplaces', 'frontage', 'fuel', 'garage_spaces', 'garage_type', 
                 'geographic_ward', 'house_extension', 'house_number', 'interior_condition', 'market_value', 
                 'market_value_date', 'number_of_bathrooms', 'number_of_bedrooms', 'number_of_rooms', 'number_stories', 
                 'other_building', 'parcel_number', 'parcel_shape','quality_grade', 'recording_date', 
                 'sale_date', 'sale_price', 'separate_utilities', 'sewer', 'site_type', 
                 'street_designation', 'street_direction', 'taxable_building', 'taxable_land', 'topography', 
                 'total_area', 'total_livable_area', 'type_heater', 'unfinished', 'unit', 
                 'utility', 'view_type', 'year_built', 'year_built_estimate', 'zip_code']
len(cols_interest)

50

In [4]:
# read in data using columns of interest 
data_step1 = data[cols_interest]
# compute percentage of missing values for each column
data_step1.isnull().sum()/len(data_step1)

assessment_date              0.940806
basements                    0.437612
building_code_description    0.000022
category_code_description    0.000241
census_tract                 0.000071
central_air                  0.505662
depth                        0.001173
exempt_building              0.000372
exempt_land                  0.000372
exterior_condition           0.047583
fireplaces                   0.007855
frontage                     0.001182
fuel                         0.973475
garage_spaces                0.008280
garage_type                  0.137328
geographic_ward              0.000071
house_extension              0.954120
house_number                 0.000000
interior_condition           0.048943
market_value                 0.000372
market_value_date            1.000000
number_of_bathrooms          0.007688
number_of_bedrooms           0.007042
number_of_rooms              0.057768
number_stories               0.007030
other_building               0.997751
parcel_numbe

### Step 2: drop columns with more than 90% missing values 

In [5]:
# drop columns with over 90% missing values
data_step2 = data_step1.drop(['assessment_date', 'fuel', 'house_extension', 'market_value_date', 'other_building', 
                              'quality_grade', 'separate_utilities', 'sewer', 'unfinished', 'unit', 
                              'utility'], axis = 1)
print(len(data_step2.columns))
print(data_step2.columns)
print(data_step2.info())

39
Index(['basements', 'building_code_description', 'category_code_description',
       'census_tract', 'central_air', 'depth', 'exempt_building',
       'exempt_land', 'exterior_condition', 'fireplaces', 'frontage',
       'garage_spaces', 'garage_type', 'geographic_ward', 'house_number',
       'interior_condition', 'market_value', 'number_of_bathrooms',
       'number_of_bedrooms', 'number_of_rooms', 'number_stories',
       'parcel_number', 'parcel_shape', 'recording_date', 'sale_date',
       'sale_price', 'site_type', 'street_designation', 'street_direction',
       'taxable_building', 'taxable_land', 'topography', 'total_area',
       'total_livable_area', 'type_heater', 'view_type', 'year_built',
       'year_built_estimate', 'zip_code'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581191 entries, 0 to 581190
Data columns (total 39 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   ----

### Step 3: transform  categorical data

In [6]:
# transform categorical data 
data_tf = data_step2.copy()

# feature 0: basements
data_tf['basements'] = data_step2['basements'].replace({'0':'None','1':'full','2':'full','3':'full','4':'full',
                                                        'A':'full', 'B': 'full','C': 'full','D': 'full','E':'partial',
                                                        'F':'partial','G': 'partial','H': 'partial','I':'full',
                                                        'J':'partial'})
print(data_tf.basements.value_counts(dropna=False))

NaN        254336
full       169102
partial    146574
None        11179
Name: basements, dtype: int64


In [7]:
# feature 1: building_code_description
m = ['MA','ST','ME','VA']
n = ['MASON','STONE','METAL','VACANT'] 
for i in range(len(m)):
    data_tf.loc[data_tf['building_code_description'].str.contains(m[i],na=False),'building_code_description'] = n[i]
data_tf.loc[(~data_tf.building_code_description.isin(n))&(data_tf.building_code_description.notnull()),
            'building_code_description'] = 'MASON'
print(data_tf.building_code_description.value_counts(dropna=False))

MASON     512256
VACANT     37054
STONE      26141
METAL       5727
NaN           13
Name: building_code_description, dtype: int64


In [8]:
# feature 2: category_code_description
print(data_tf.category_code_description.value_counts(dropna=False))

Single Family    462056
Vacant Land       43845
Multi Family      42484
Mixed Use         14347
Commercial        14004
Industrial         4315
NaN                 140
Name: category_code_description, dtype: int64


In [9]:
# feature 3: census_tract, can only run once
data_tf['census_tract'].fillna(0, inplace=True)
data_tf['census_tract'] = data_tf['census_tract'].astype('int')
data_tf.loc[(data_tf['census_tract']>0)&(data_tf['census_tract']<=300), 'census_tract'] = 1
data_tf.loc[(data_tf['census_tract']>=301)&(data_tf['census_tract']<=600),'census_tract'] = 2
data_tf.loc[data_tf['census_tract']>=601, 'census_tract'] = 3
data_tf['census_tract'] = data_tf['census_tract'].replace({0:np.NaN,1:'A',2:'B',3:'C'})

In [10]:
print(data_tf.census_tract.value_counts(dropna=False))

A      433021
B      147212
C         917
NaN        41
Name: census_tract, dtype: int64


In [11]:
# feature 4: central_air
data_tf['central_air'] = data_step2['central_air'].replace({'0': 'N', '1': 'Y','Y': 'Y', 'N': 'N'})
print(data_tf.central_air.value_counts(dropna=False))

NaN    293886
N      174067
Y      113238
Name: central_air, dtype: int64


In [12]:
# feature 8: exterior_condition
data_tf['exterior_condition'] = data_step2['exterior_condition'].replace({0:'none',1:'new',2:'rehabilitated',
                                                                          3:'above average',4:'rehabilitated',5:'average', 
                                                                          6: 'below average',7:'vacant',8:'sealed',
                                                                          9:'compromised'})
print(data_tf.exterior_condition.value_counts(dropna=False))

rehabilitated    450108
above average     43662
NaN               27655
none              19411
average           17679
new               13770
vacant             5702
below average      3204
Name: exterior_condition, dtype: int64


In [13]:
# feature 12: garage_type
data_tf['garage_type'] = data_step2['garage_type'].replace({'A':'attached', 'B':'attached','C':'detached',
                                                            'F':'attached', 'S':'attached', 'T':'detached',
                                                            '0':'none', '0.0':'none', 0.0:'none',
                                                            '1':'attached', '1.0':'attached', 1.0:'attached',
                                                            '2':'attached', '2.0':'attached', 2.0:'attached',
                                                            '3':'attached', '3.0':'attached', 3.0:'attached'})
print(data_tf.garage_type.value_counts(dropna=False))

none        305799
attached    176718
NaN          79814
detached     18860
Name: garage_type, dtype: int64


In [14]:
# feature 13: geographic_ward, can only run once
data_tf['geographic_ward'].fillna(0, inplace=True)
data_tf['geographic_ward'] = data_tf['geographic_ward'].astype('int')
data_tf.loc[(data_tf['geographic_ward']>0)&(data_tf['geographic_ward']<=30), 'geographic_ward'] = 1
data_tf.loc[(data_tf['geographic_ward']>=31)&(data_tf['geographic_ward']<=50),'geographic_ward'] = 2
data_tf.loc[data_tf['geographic_ward']>=51, 'geographic_ward'] = 3
data_tf['geographic_ward'] = data_tf['geographic_ward'].replace({0:np.NaN,1:'A',2:'B',3:'C'})

In [15]:
data_tf.geographic_ward.value_counts(dropna=False)

A      241479
B      193817
C      145854
NaN        41
Name: geographic_ward, dtype: int64

In [16]:
# feature 14: house_number
data_tf['house_number'] = np.where(data_step2['house_number']%2==0, 'South or West', 'North or East')

In [17]:
data_tf.house_number.value_counts(dropna=False)

South or West    293063
North or East    288128
Name: house_number, dtype: int64

In [18]:
# feature 15: interior_condition
data_tf['interior_condition'] = data_step2['interior_condition'].replace({0:'none',1:'new',2:'new',
                                                                          3:'above average',4:'average', 
                                                                          5:'below average',6:'vacant',7:'sealed'})
print(data_tf.exterior_condition.value_counts(dropna=False))

rehabilitated    450108
above average     43662
NaN               27655
none              19411
average           17679
new               13770
vacant             5702
below average      3204
Name: exterior_condition, dtype: int64


In [19]:
# feature 21: parcel_number, can run only once
data_tf['parcel_number'].fillna(0, inplace=True)
data_tf['parcel_number'] = data_tf['parcel_number'].astype('int')
data_tf.loc[(data_tf['parcel_number']>0)&(data_tf['parcel_number']<=4e8),'parcel_number'] = 1
data_tf.loc[(data_tf['parcel_number']>=4e8+1)&(data_tf['parcel_number']<=7e8),'parcel_number'] = 2
data_tf.loc[data_tf['parcel_number']>=7e8+1,'parcel_number'] = 3
data_tf['parcel_number'] = data_tf['parcel_number'].replace({0:np.NaN,1:'A',2:'B',3:'C'})

In [20]:
print(data_tf.parcel_number.value_counts(dropna=False))

A    279344
B    220468
C     81379
Name: parcel_number, dtype: int64


In [21]:
# feature 22: parcel_shape
data_tf['parcel_shape'] = data_step2['parcel_shape'].replace({' ':np.NaN})
print(data_tf.parcel_shape.value_counts(dropna=False))

E      528235
A       38513
NaN      6590
B        6309
C        1478
D          66
Name: parcel_shape, dtype: int64


In [22]:
# feature 26: site_type
data_tf['site_type'].value_counts(dropna=False)

NaN    285033
A      257318
B       37545
D         638
C         390
E         204
G          33
F          30
Name: site_type, dtype: int64

In [23]:
# feaure 27: street_designation
data_tf['street_designation'].value_counts(dropna=False)

ST      426727
AVE      91130
RD       31076
LA        8201
DR        5920
PL        5719
BLV       2765
SQ        2417
TER       1614
WAY       1353
CT        1204
CIR        805
LN         609
PK         583
PKY        377
PLZ        168
BLVD       145
MEW         79
ALY         74
WLK         67
ML          47
PIKE        31
HTS         24
PTH         12
PKWY        10
ROW          9
MEWS         7
WALK         6
NaN          5
PATH         3
MALL         3
EXP          1
Name: street_designation, dtype: int64

In [24]:
# feature 28: street_direction   
data_tf['street_direction'].value_counts(dropna=False)

NaN    354729
N       96431
S       58525
W       37234
E       34272
Name: street_direction, dtype: int64

In [25]:
# feature 31: topography      
data_tf['topography'] = data_step2['topography'].replace({'0':'None'})
data_tf['topography'].value_counts(dropna=False)

F       506514
NaN      38882
A        30412
E         4816
B          256
C          174
D          114
None        23
Name: topography, dtype: int64

In [26]:
# feature 34: type_heater  
data_tf['type_heater'] = data_step2['type_heater'].replace({'0':'None'})
data_tf['type_heater'].value_counts(dropna=False)

NaN     285933
H       125267
A        97967
B        60880
G         4916
C         2979
None      1907
E          755
D          587
Name: type_heater, dtype: int64

In [27]:
# feature 35: view_type 
data_tf['view_type'] = data_step2['view_type'].replace({'0':'None'})
data_tf['view_type'].value_counts(dropna=False)

I       521733
NaN      20996
A        15277
C         7613
None      4965
D         4018
H         2738
E         2081
B         1770
Name: view_type, dtype: int64

In [28]:
# feature 37: year_built_estimate
data_tf['year_built_estimate'] = data_step2['year_built_estimate'].replace({'0':'N'})
data_tf['year_built_estimate'].value_counts(dropna=False)

Y      438150
NaN    142747
N         294
Name: year_built_estimate, dtype: int64

In [29]:
# feature 38: zip_code, can only run once
data_tf['zip_code'].fillna(0, inplace=True)
data_tf['zip_code'] = data_tf['zip_code'].astype('int')
data_tf.loc[(data_tf['zip_code']>0)&(data_tf['zip_code']<=19120),'zip_code'] = 1
data_tf.loc[(data_tf['zip_code']>=19121)&(data_tf['zip_code']<=19140),'zip_code'] = 2
data_tf.loc[data_tf['zip_code']>=19141,'zip_code'] = 3
data_tf['zip_code'] = data_tf['zip_code'].replace({0:np.NaN,1:'A',2:'B',3:'C'})

In [30]:
data_tf['zip_code'].value_counts(dropna=False)

B      259902
C      202936
A      118314
NaN        39
Name: zip_code, dtype: int64

### Step 4: transform time-related data

In [31]:
# feature 23: recording_date, can only run once
data_tf['recording_date'] = pd.to_datetime(data_tf['recording_date'], errors='coerce')
data_tf['recording_date'] = data_tf['recording_date'] - data_tf['recording_date'].min()
data_tf['recording_date'].fillna(pd.Timedelta(seconds=0), inplace=True)
data_tf['recording_date'] = data_tf['recording_date'].astype('str').apply(lambda x:x[:-5]).astype('int64')
data_tf.loc[data_tf['recording_date']==0,'recording_date'] = np.NaN

In [32]:
# feature 24: sale_date, can only run once
data_tf['sale_date'] = data_tf['sale_date'].replace({'2200-01-17 00:00:00':'2022-01-17 00:00:00'})
data_tf['sale_date'] = pd.to_datetime(data_tf['sale_date'], errors='coerce')
data_tf['sale_date'] = data_tf['sale_date'] - data_tf['sale_date'].min()
data_tf['sale_date'].fillna(pd.Timedelta(seconds=0), inplace=True)
data_tf['sale_date'] = data_tf['sale_date'].astype('str').apply(lambda x:x[:-5]).astype('int64')
data_tf.loc[data_tf['sale_date']==0,'sale_date'] = np.NaN

In [33]:
# feature 36: year_built  
data_tf['year_built'] = data_tf['year_built'].replace({'196Y':'1965',0:np.NaN})
data_tf['year_built'] = 2022 - data_tf['year_built'].astype(float)

In [34]:
print(data_tf.dtypes)
data_tf.head()

basements                     object
building_code_description     object
category_code_description     object
census_tract                  object
central_air                   object
depth                        float64
exempt_building              float64
exempt_land                  float64
exterior_condition            object
fireplaces                   float64
frontage                     float64
garage_spaces                float64
garage_type                   object
geographic_ward               object
house_number                  object
interior_condition            object
market_value                 float64
number_of_bathrooms          float64
number_of_bedrooms           float64
number_of_rooms              float64
number_stories               float64
parcel_number                 object
parcel_shape                  object
recording_date               float64
sale_date                    float64
sale_price                   float64
site_type                     object
s

,basements,building_code_description,category_code_description,census_tract,central_air,depth,exempt_building,exempt_land,exterior_condition,fireplaces,...,taxable_building,taxable_land,topography,total_area,total_livable_area,type_heater,view_type,year_built,year_built_estimate,zip_code
0,NaN,VACANT,NaN,A,NaN,NaN,100.0,0.0,NaN,NaN,...,0.0,0.0,F,NaN,NaN,NaN,I,NaN,NaN,C
1,NaN,VACANT,Multi Family,A,NaN,NaN,29400.0,0.0,NaN,NaN,...,0.0,0.0,F,NaN,NaN,NaN,I,NaN,NaN,C
2,NaN,VACANT,NaN,A,NaN,NaN,14200.0,0.0,NaN,NaN,...,0.0,0.0,F,NaN,NaN,NaN,I,NaN,NaN,C
3,NaN,VACANT,Vacant Land,A,NaN,NaN,0.0,5200.0,NaN,NaN,...,0.0,0.0,F,681.0,NaN,NaN,I,NaN,NaN,C
4,NaN,VACANT,Vacant Land,A,NaN,NaN,0.0,21000.0,NaN,NaN,...,0.0,0.0,F,712.0,NaN,NaN,I,NaN,NaN,C


### Step 5: split data into training, validation and test set

In [87]:
Data_all = data_tf.copy()

X_data = Data_all.drop(['market_value','sale_price'], axis=1)
y_data = Data_all.market_value

# divide data into training, validation and test set
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=123)
X_train_train, X_train_val, y_train_train, y_train_val = train_test_split(X_train, y_train, test_size=0.2, random_state=123)

In [136]:
# save files for EDA
X_train_train_market_sale_price = pd.concat([y_train_train, Data_all.loc[X_train_train.index, 'sale_price']], axis=1)
X_train_train_eda = pd.concat([X_train_train, X_train_train_market_sale_price], axis=1)
X_train_train_eda.to_csv('X_train_train_eda.csv')

### Step 6: preprocess training data 

In [ ]:
# for categorical data, fill missing values with mode and then one-hot encode
cat_features = ['basements', 'building_code_description', 'category_code_description','census_tract', 'central_air', 
                'exterior_condition', 'garage_type', 'geographic_ward', 'house_number','interior_condition', 
                'parcel_number', 'parcel_shape', 'site_type', 'street_designation', 'street_direction',
                'topography', 'type_heater', 'view_type', 'year_built_estimate', 'zip_code']
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore',sparse=False))
])

# for numerical data, fill missing values with median and then scale
num_features = ['depth', 'exempt_building', 'exempt_land','fireplaces', 'frontage', 
                'garage_spaces', 'number_of_bathrooms','number_of_bedrooms', 'number_of_rooms','number_stories', 
                'recording_date','sale_date','taxable_building', 'taxable_land', 'total_area', 
                'total_livable_area','year_built']
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')), 
    ('scaler', StandardScaler())
])

# Combine two transformers into single ColumnTransformer preprocessor
preprocessor = ColumnTransformer([
    ('cat', cat_pipe, cat_features),
    ('num', num_pipe, num_features)
])

In [128]:
# use pipeline to fit training data
preprocessor_train = preprocessor.fit(X_train_train)

# prepare column names
cat_columns = preprocessor.named_transformers_['cat']['encoder'].get_feature_names(cat_features)
columns = np.append(cat_columns, num_features)

# preprocess training data
X_train_train_final = pd.DataFrame(preprocessor_train.transform(X_train_train), index=X_train_train.index, columns=columns)
y1_train_train_final = y_train_train.fillna(y_train_train.median())
y2_train_train = Data_all.loc[X_train_train.index, 'sale_price']
y2_train_train_final = y2_train_train.fillna(y2_train_train.median())

In [131]:
# save training data to file
X_train_train_final.to_csv('X_train_train_model.csv')
y1_train_train_final.to_csv('y1_train_train_model.csv')
y2_train_train_final.to_csv('y2_train_train_model.csv')

### Step 7: preprocess validation and test data 

In [132]:
# preprocess validation data
X_train_val_final = pd.DataFrame(preprocessor_train.transform(X_train_val), index=X_train_val.index, columns=columns)
y1_train_val_final = y_train_val.fillna(y_train_train.median())
y2_train_val = Data_all.loc[X_train_val.index, 'sale_price']
y2_train_val_final = y2_train_val.fillna(y2_train_train.median())

# preprocess test data
X_test_final = pd.DataFrame(preprocessor_train.transform(X_test), index=X_test.index, columns=columns)
y1_test_final = y_test.fillna(y_train_train.median())
y2_test = Data_all.loc[X_test.index, 'sale_price']
y2_test_final = y2_test.fillna(y2_train_train.median())

In [134]:
# save validation data to file
X_train_val_final.to_csv('X_train_val_model.csv')
y1_train_val_final.to_csv('y1_train_val_model.csv')
y2_train_val_final.to_csv('y2_train_val_model.csv')

In [135]:
# save test data to file
X_test_final.to_csv('X_test_model.csv')
y1_test_final.to_csv('y1_test_model.csv')
y2_test_final.to_csv('y2_test_model.csv')

In [146]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
lr = LinearRegression().fit(X_train_train_final, y1_train_train_final)
mean_squared_error(lr.predict(X_test_final), y1_test_final)

349793.2800060819